In [1]:
import urllib2
import cookielib
from bs4 import BeautifulSoup
import html5lib
import pandas as pd

url = 'http://orpts.tax.ny.gov/cfapps/MuniPro/muni_theme/muni/ratehistory.cfm?swis=012200'

In [32]:
def load_swis(s): 
    if len(str(s)) == 5:
        swis = '0' + str(s)
    else:
        swis = str(s)
    cj = cookielib.CookieJar()
    opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(cj))
    baseurl = "http://orpts.tax.ny.gov/cfapps/MuniPro/muni_theme/county/equasearch.cfm?swis=01"
    regex = '<ul class="w462">(.*?)</ul>'

    opener.open(baseurl)
    urllib2.install_opener(opener)

    request = urllib2.Request('http://orpts.tax.ny.gov/cfapps/MuniPro/muni_theme/muni/ratehistory.cfm?swis=' + swis)
    request.add_header('Referer', baseurl)

    requestData = urllib2.urlopen(request)
    page = requestData.read()
    starting_pos = page.find('<table summary="Equalization Rate History"')
    eq_table = pd.read_html(page[starting_pos:], flavor='bs4', header=0)
    return eq_table

soup = BeautifulSoup(page[starting_pos:])

for tr in soup.find_all('tr'):
    tds = tr.find_all('td')
    print "Nome: %s, Cognome: %s, Email: %s" % \
          (tds[0].text, tds[0].text, tds[0].text)

In [5]:
# eq_table[0].iloc[:,:4].to_csv('010300.csv', index=False)

In [28]:
swis = pd.read_csv('SWISCodes.csv')['SWIS Code']

In [58]:
merged_table = pd.DataFrame()
for s in swis:
    print(str(s))
    output_table = load_swis(s)[0]
    output_table['swis'] = s
    merged_table = merged_table.append(output_table)

10000
10100
10300
11800
12000
12200
12400
12401
12600
12601
12603
12800
12801
13000
13001
13200
13400
13401
13600
13800
20000
22000
22001
22200
22400
22600
22601
22800
22801
23000
23001
23200
23201
23400
23600
23800
23801
23803
24000
24001
24200
24400
24600
24800
24801
25000
25001
25200
25400
25600
25800
25801
26000
26200
26400
26600
26800
27000
27001
27200
27400
27600
30000
30200
32000
32200
32400
32600
32800
33000
33001
33200
33400
33600
33601
33800
34000
34200
34201
34400
34401
34600
34601
34603
34800
35000
35001
40000
41200
41600
42000
42001
42200
42400
42401
42600
42800
42801
43000
43001
43200
43400
43600
43601
43800
44000
44001
44200
44400
44600
44800
45000
45200
45400
45401
45600
45800
46000
46200
46400
46401
46600
46800
47000
47001
47200
47201
47400
47401
47600
47601
47800
48000
48200
48400
48401
50000
50100
52000
52001
52200
52201
52400
52401
52403
52600
52800
53000
53200
53400
53401
53600
53800
53801
54000
54200
54201
54400
54600
54800
55000
55200
55400
55401
55600
55601
5580

ValueError: No tables found

In [62]:
merged_table[0:60000].to_excel('merged_part1.xls')

In [63]:
merged_table[60000:].to_excel('merged_part2.xls')